# Analizar Datos de Formulario

In [ ]:
# PARA GOOGLE COLAB
#!pip -q install gspread
#!pip install rapidfuzz
#!pip install autocorrect
#!pip install unidecode
#!pip install pyspellchecker==0.7.2
#!pip install nltk
#!pip install wordcloud

In [ ]:
# PARA GOOGLE COLAB
#import pandas as pd
#from google.colab import auth
#import google.auth
#import gspread
#import pandas as pd
#from rapidfuzz import process
#import matplotlib.pyplot as plt

In [ ]:
# PARA ENTORNO LOCAL
import pandas as pd
from rapidfuzz import process
import matplotlib.pyplot as plt

In [ ]:
# PARA GOOGLE COLAB
# Autenticarse y leer Google Sheets con gspread
#auth.authenticate_user()

#creds, _ = google.auth.default()
#gc = gspread.authorize(creds)

# Form de respuestas
#SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/URL_PROPIO/edit?usp=drive_link"

# Enlazando con spreadsheet
#sh = gc.open_by_url(SPREADSHEET_URL)
#ws = sh.sheet1

# Leyendo datos
#df = pd.DataFrame(ws.get_all_records())

# Eliminando columnas con información sensible
#df.drop(columns=["Dirección de correo electrónico", "Nombre completo"], inplace=True)

# FIN PARA GOOGLE COLAB

# Nombres

df = pd.read_excel("../data/C1S1-Form.xlsx")

df.head()

In [ ]:
#Renombrando columnas
rename_dic = {
    "Marca temporal": "Timestamp",
    "¿Qué carrera estudias?": "Carrera",
    "¿En qué año entraste a la universidad?": "Carnet",
    "¿En qué semestre de tu carrera te encuentras?": "Semestre",
    "¿Tienes intención de obtener el diploma de participación en el primer curso impartido por el club? (Requiere la asistencia a un mínimo de 4 sesiones así como la presentación final de un mini proyecto.)": "Diploma",
    "¿Tienes conocimiento previo sobre programación?" : "C-Programación",
    "¿Tienes conocimiento previo sobre ciencia o ingeniería de datos?": "C-DSE",
    "¿Cuáles son tus motivaciones para querer participar del Club de Ciencia de Datos e Ingeniería de Datos?": "Motivación"
}

df = df.rename(columns=rename_dic)
df.head()


## Estandarizando Carrera

In [ ]:
# Todo en minúsculas
df['Carrera'] = df['Carrera'].str.lower()
#df['Carrera'] = df['Carrera'].apply(lambda x: x.lower())

# Removiendo palabras inecesarias
to_remove = ["ingeniería","ingenieria","ing.", ".", "en"]

def remove_words(text, words):
    for word in words:
        text = text.replace(word, "").strip()
    return text

df['Carrera'] = df['Carrera'].apply(lambda x: remove_words(x, to_remove))

df.head()

In [ ]:
# Carreras únicas
df['Carrera'].unique()

In [ ]:
# Corrigiendo carreras
key_words = ["industrial", "ciencias y sistemas", "química", "mecánica eléctrica", "eléctrica", "civil", "maestría", "electrónica", "mecánica industrial", "mecánica", "física"]


# Función para obtener la palabra más cercana
def asignar_keyword(texto, key_words):
    match, score, _ = process.extractOne(texto, key_words)
    return match if score > 70 else None  # se puede ajustar el umbral


df["Carrera_Ok"] = df["Carrera"].apply(lambda x: asignar_keyword(x, key_words))
df.head()


In [ ]:
# Conteo y gráfico
ax = df["Carrera_Ok"].value_counts().plot.bar(
    figsize=(6,4),
    color="#20B9DB",   # club_lightblue
    edgecolor="black"
)

# Título y etiquetas
plt.title("Distribución de Carrera_Ok")
plt.xlabel("Carrera")
plt.ylabel("Frecuencia")
plt.xticks(rotation=45, ha="right")

# Mostrar número arriba de cada barra
for p in ax.patches:
    ax.annotate(
        str(p.get_height()),
        (p.get_x() + p.get_width() / 2, p.get_height()),
        ha="center", va="bottom", fontsize=9
    )

# Quitar márgenes superior y derecho
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
plt.show()

## Analizando Lenguaje

In [ ]:
import re
from unidecode import unidecode
from spellchecker import SpellChecker
from autocorrect import Speller
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud

In [ ]:
nltk.download('stopwords')

In [ ]:
# Limpieza básica
col_texto = "Motivación"
df_proc = df[[col_texto]].dropna().copy()

In [ ]:
# Pasar a minúsculas
df_proc["texto_lower"] = df_proc[col_texto].astype(str).str.lower()

In [ ]:
# Corrección ortográfica en español
spell = Speller(lang='es')
df_proc["texto_corr"] = df_proc["texto_lower"].apply(spell)
df_proc[["texto_lower", "texto_corr"]].head(10)

In [ ]:
#Mantener símbolos útiles
pat = re.compile(r"[^a-záéíóúüñ\s]")
df_proc["texto_clean"] = df_proc["texto_corr"].str.replace(pat, " ", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
df_proc["tokens"] = df_proc["texto_clean"].str.split()
df_proc[["texto_corr", "texto_clean"]].head(10)

In [ ]:
# Eliminando stopwords
stop_es = set(stopwords.words('spanish'))
df_proc["tokens_nostop"] = df_proc["tokens"].apply(lambda toks: [t for t in toks if t not in stop_es and len(t) > 2])
df_proc[["texto_clean", "tokens_nostop"]].head(10)

In [ ]:
# Creando Corpus
corpus = " ".join([" ".join(toks) for toks in df_proc["tokens_nostop"]])
corpus

In [ ]:
# Wordcloud
from wordcloud import WordCloud

wc = WordCloud(
    width=1000,
    height=600,
    background_color="white",
    collocations=False
).generate(corpus)

plt.figure(figsize=(8, 4))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
#plt.title("Nube de palabras — Motivación")
plt.show()